In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1. Load data (interpretable features: e.g. mean radius, texture, smoothness…)
data = load_breast_cancer()
X, y = data.data, data.target

# 2. Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 3. Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

# 4. Convert to torch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test  = torch.tensor(X_test, dtype=torch.float32)
y_test  = torch.tensor(y_test, dtype=torch.long)

# 5. Create DataLoaders
train_ds = TensorDataset(X_train, y_train)
test_ds  = TensorDataset(X_test,  y_test)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=32, shuffle=False)

# 6. Define the neural network
class BreastCancerNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 2)           # binary classification
        )
    def forward(self, x):
        return self.model(x)

# 7. Instantiate model, loss fn, optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BreastCancerNet(input_dim=X_train.shape[1]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

# 8. Training loop
num_epochs = 20
for epoch in range(1, num_epochs+1):
    model.train()
    train_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)

        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * xb.size(0)

    train_loss /= len(train_loader.dataset)

    # Evaluation
    model.eval()
    correct = 0
    with torch.no_grad():
        for xb, yb in test_loader:
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb)
            preds = torch.argmax(logits, dim=1)
            correct += (preds == yb).sum().item()

    test_acc = correct / len(test_loader.dataset)
    print(f"Epoch {epoch:02d} | Train Loss: {train_loss:.4f} | Test Acc: {test_acc:.4f}")


Epoch 01 | Train Loss: 0.6587 | Test Acc: 0.9123
Epoch 02 | Train Loss: 0.4943 | Test Acc: 0.9386
Epoch 03 | Train Loss: 0.3106 | Test Acc: 0.9386
Epoch 04 | Train Loss: 0.1871 | Test Acc: 0.9386
Epoch 05 | Train Loss: 0.1347 | Test Acc: 0.9474
Epoch 06 | Train Loss: 0.1030 | Test Acc: 0.9474
Epoch 07 | Train Loss: 0.0827 | Test Acc: 0.9386
Epoch 08 | Train Loss: 0.0759 | Test Acc: 0.9474
Epoch 09 | Train Loss: 0.0640 | Test Acc: 0.9386
Epoch 10 | Train Loss: 0.0686 | Test Acc: 0.9561
Epoch 11 | Train Loss: 0.0637 | Test Acc: 0.9561
Epoch 12 | Train Loss: 0.0573 | Test Acc: 0.9649
Epoch 13 | Train Loss: 0.0552 | Test Acc: 0.9649
Epoch 14 | Train Loss: 0.0549 | Test Acc: 0.9649
Epoch 15 | Train Loss: 0.0452 | Test Acc: 0.9474
Epoch 16 | Train Loss: 0.0460 | Test Acc: 0.9649
Epoch 17 | Train Loss: 0.0449 | Test Acc: 0.9649
Epoch 18 | Train Loss: 0.0422 | Test Acc: 0.9561
Epoch 19 | Train Loss: 0.0426 | Test Acc: 0.9561
Epoch 20 | Train Loss: 0.0336 | Test Acc: 0.9561
